## Network Attack Example

### TDA Analysis Dr RI Young Feb 2020

#### Model of a network attack.  Attack is propagation of an infectious desease.  Graph model is based upon  networkX.  Key to this is graph shape extracted as points for scatter graph.  This is ripe for topological analysis, hopefully.

I've added the ability for nodes to recover from infection. To maintain the network context, this could represent someone realising their machine is infected and updating/installing antivirus to counteract it.

#### Network attack from http://cody.bunta.in/assets/classes/2018_spring_umd_enpm809g/Class10/DiffusionExamples.html

In [ ]:
%matplotlib inline

In [1]:
#Imports
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from celluloid import Camera

from JSAnimation.IPython_display import display_animation, anim_to_html

from scipy import signal
from scipy.misc import electrocardiogram
from scipy.signal import find_peaks

import math

from itertools import product

from sklearn import datasets
from sklearn.decomposition import PCA
from scipy.stats import multivariate_normal as mvn

from ripser import Rips

import random as rd

In [ ]:
#Class for plotting evolving infection and its persistent homology at each step.
class infectionGrapher():
    
    def __init__(self,graph): #Initialization parameters
        self.g = graph
        self.n = len(self.g.nodes())
        self.pos = nx.get_node_attributes(self.g, "pos")
        self.infected_map = {x:False for x in self.g.nodes}
        self.recovered_map = {x:False for x in self.g.nodes}
        self.min_dur_map = {x:True for x in self.g.nodes}
        self.graphPlots = []
        self.myInfecData = []
        self.myRecovData = []
        self.rips = Rips()
        self.data_healthy = []
        self.data_infected = []
        self.data_recovered = []
        self.data_uninfected = []

    def reset(self): #In case you want to clear the state without resetting the graph
        self.infected_map = {x:False for x in self.g.nodes}
        self.recovered_map = {x:False for x in self.g.nodes}
        self.min_dur_map = {x:True for x in self.g.nodes}
        self.ims = []
        self.myInfecData = []
        self.myRecovData = []        
        self.data_healthy = []
        self.data_infected = []
        self.data_recovered = []
        self.data_uninfected = []
        
    def initialInfection(self,case_zero): #Starting condition - to-do: extend to multiple case_zeros 
        self.infected_map[case_zero] = True
        self.min_dur_map[case_zero] = False
        
    def infectionStep(self,trans_p,rec_p,rec_trans_p,vacc_time,step_number): #One step of the infection loop
        # For each infected node, find and infect uninfected neighbors
        for ill_node in [n for n, i in self.infected_map.items() if i]:
            for neighbor in self.g.neighbors(ill_node):

                # probabilistic traaccessing (<matplotlib.collections.PathCollectionnsmission
                if (self.recovered_map[neighbor] == False and self.infected_map[neighbor] == False):
                    if ( np.random.random() < trans_p ):
                        self.infected_map[neighbor] = True
                        self.recovered_map[neighbor] = False
                        self.min_dur_map[neighbor] = False #This ensures they don't recover immediately in the next loop
                if (self.recovered_map[neighbor] == True): #For recovery immunity assumptions
                    if ( np.random.random() < rec_trans_p ):
                        self.infected_map[neighbor] = True
                        self.recovered_map[neighbor] = False
                        self.min_dur_map[neighbor] = False

        # For each previously infected node, see if it recovers
        for ill_node in [n for n, i in self.infected_map.items() if i]:
            if ( np.random.random() < rec_p  and self.min_dur_map[ill_node] == True):
                self.infected_map[ill_node] = False
                self.recovered_map[ill_node] = True

        # Update infected colors
        if (vacc_time == None): #If we're not using the vaccination feature
            self.colors = [1.0 if self.infected_map[x] else 0.4 if self.recovered_map[x] else 0.0 for x in self.g.nodes]
        else:
            if (step_number < vacc_time): #if vaccine hasn't been introduced yet
                self.colors = [1.0 if self.infected_map[x] else 0.4 if self.recovered_map[x] else 0.0 for x in self.g.nodes]
            if (step_number >= vacc_time): #post vaccination
                self.colors = [1.0 if self.infected_map[x] else 0.75 if self.recovered_map[x] else 0.0 for x in self.g.nodes]
                
        #Set all the ill nodes as being capable of recovery
        for ill_node in [n for n, i in self.infected_map.items() if i]:
            self.min_dur_map[ill_node] = True

    def initialGraph(self): #Draw the dirst graph
        self.graph_fig = plt.figure()
        self.graph_fig.set_figheight(10)
        self.graph_fig.set_figwidth(15)
        self.ax = self.graph_fig.gca()
        nx.draw_networkx_edges(self.g, self.pos, ax=self.ax)
        
        self.cmap = plt.get_cmap("rainbow")
        self.colors = [1.0 if self.infected_map[x] else 0.0 for x in self.g.nodes]
        
        self.graphPlots = [(nx.draw_networkx_nodes(self.g, self.pos, node_color=self.colors, cmap=self.cmap, ax=self.ax, vmin=0.0, vmax=1.0),)]
    
    def updateGraph(self): #Draw a graph at the current state
        self.graphPlots.append(
        (nx.draw_networkx_nodes(self.g, self.pos, node_color=self.colors, cmap=self.cmap, ax=self.ax, vmin=0.0, vmax=1.0),)
        )
        
    def infector(self,niters,case_zero,trans_p,rec_p=0.0,rec_trans_p=0.0,vacc_time=None, 
                 draw_graph = True,plot_homology = True): #Runs the infection loop
        
        #Starting case
        self.initialInfection(case_zero)
        
        #If we're drawing graphs
        if (draw_graph == True): 
            self.initialGraph()
            
        #If we're plotting homologies
        if (plot_homology == True):
                self.data_healthy.append(np.transpose(self.healthyCounter()))
                self.data_infected.append(np.transpose(self.infectedCounter()))
                self.data_recovered.append(np.transpose(self.recoveredCounter()))
                self.data_uninfected.append(np.transpose(self.uninfectedCounter()))
        
        #Count current infections
        last_infected_count = sum([1 for n, i in self.infected_map.items() if i])
        infected_counts = [last_infected_count]

        #Count current recoveries
        last_recovered_count = sum([1 for n, i in self.recovered_map.items() if i])
        recovered_counts = [last_recovered_count]
        
        for i in range(niters): #Iterate over the number of steps
            if (vacc_time != None): #If we're in a post-vaccination development world
                if (i >= vacc_time):
                    rec_trans_p = 0.0 #Recovered nodes can't be reinfected
            
            self.infectionStep(trans_p,rec_p,rec_trans_p,vacc_time,i) #Run a step of the infection
            
            #Add current graph if we're drawing
            if (draw_graph == True):
                self.updateGraph()
                
            #Add current node data if we're drawing homologies
            if (plot_homology == True):
                self.data_healthy.append(np.transpose(self.healthyCounter()))
                self.data_infected.append(np.transpose(self.infectedCounter()))
                self.data_recovered.append(np.transpose(self.recoveredCounter()))
                self.data_uninfected.append(np.transpose(self.uninfectedCounter()))
             
            #Update infected and recovered counts
            infected_count = sum([1 for n, i in self.infected_map.items() if i])
            recovered_count = sum([1 for n, i in self.recovered_map.items() if i])
    
            #Update list of total infections and recoveries
            self.myInfecData.append(infected_count)
            self.myRecovData.append(recovered_count)
    
            #If we've reached total infection, stop
            if ( infected_count == len(self.g.nodes) ):
                print("Total Infection After:", i+1)
                break
        
            #If there are no more infected cases, stop
            if ( infected_count == 0 ):
                print("Total recovery after:", i+1)
                break
   
            #Update infected and recovery counts
            last_infected_count = infected_count 
            infected_counts.append(last_infected_count)
            last_recovered_count = recovered_count 
            recovered_counts.append(last_recovered_count)
            
        self.fin_number = i+1 #Note when we stop
    
    ##################
    #The counter functions give the coordintes of the points with each status at each time step
    def healthyCounter(self):
        healthy0 = []
        healthy1 = []
        
        for i in range(0,self.n):
            if self.recovered_map[i] == True:
                healthy0.append(pos[i][0])
                healthy1.append(pos[i][1])
            if (self.infected_map[i] == False and self.recovered_map[i] == False): #Never infected in the first place
                healthy0.append(pos[i][0]) 
                healthy1.append(pos[i][1])
        
        return [healthy0, healthy1]
        
    def infectedCounter(self):
        infec0 = []
        infec1 = []
        
        for i in range(0,self.n):
            if self.infected_map[i] == True:
                infec0.append(pos[i][0])
                infec1.append(pos[i][1])
        
        return [infec0, infec1]
        
    def recoveredCounter(self):
        recov0 = []
        recov1 = []
        
        for i in range(0,self.n):
            if self.recovered_map[i] == True:
                recov0.append(pos[i][0])
                recov1.append(pos[i][1])        
                
        return [recov0, recov1]
            
    def uninfectedCounter(self):
        uninfected0 = []
        uninfected1 = []
        
        for i in range(0,self.n):        
            if (self.infected_map[i] == False and self.recovered_map[i] == False): #Never infected in the first place
                uninfected0.append(pos[i][0])
                uninfected1.append(pos[i][1])
                
        return [uninfected0, uninfected1]
    
    ##################

    #Animate the evolution of infection
    def graphAnimator(self):
        self.graph_ani = animation.ArtistAnimation(self.graph_fig, self.graphPlots, interval=150, repeat_delay=3000,
                                   blit=True)
        #display_animation(im_ani) #For some reason it won't do anything if you tell it to just show it.
                                    #Put this line outside the class function (referring to the class's.im_ani property)
        
    #Plot the number of infections over time
    def infectionPlotter(self):
        plt.plot(np.maximum(np.diff(self.myInfecData),0))
        plt.title('Rate of infection')
        plt.xlabel('Epoch')
        plt.ylabel('Number of new infections')
        plt.show()
        
    #Plot the number of recoveries over time
    def recoveryPlotter(self):
        plt.plot(np.maximum(np.diff(self.myRecovData),0))
        plt.title('Rate of Recovery')
        plt.xlabel('Epoch')
        plt.ylabel('Number of new recoveries')
        plt.show()
        
    #Chooses the data type based on what you want plotted for homology plotting
    def dataSelecter(self,num_type): #num_type: 0 = Healthy, 1 = Infected, 2 = Recovered, 3 = Uninfected
        data = []
        
        if num_type == 0:
            data = self.data_healthy
            label = 'healthy'
        if num_type == 1:
            data = self.data_infected
            label = 'infected'
        if num_type == 2:
            data = self.data_recovered
            label = 'recovered'
        if num_type == 3:
            data = self.data_uninfected
            label = 'uninfected'
            
        return data, label
        
    #Plots a single case of the persistence diagram
    def homologyPlot(self,num_type,step):
        data = []
        dgms = []
        
        data = self.dataSelecter(num_type)
        
        dgms = self.rips.fit_transform(data[0][step]) 
        self.rips.plot(dgms, legend=True, show=False)
        
    #Animates the persistence diagram over all iterations
    def homologyAnimator(self,num_type,file_name=None): 
        data = []
        dgms = []
        
        data = self.dataSelecter(num_type)
        
        if (file_name == None):
            file_name = "{}_persistent_homology.gif".format(data[1])
        
        pers_fig = plt.figure()
        camera = Camera(pers_fig)
        
        for i in range(self.fin_number):
            if (data[0][i].size == 0):
                camera.snap()
            else: 
                dgms = self.rips.fit_transform(data[0][i]) 
                self.rips.plot(dgms, legend=False, show=False)
                camera.snap()
            
        animation = camera.animate()
        animation.save(file_name, writer = 'imagemagick')
        

In [ ]:
n = 256
# g = nx.erdos_renyi_graph(n, 1.)
# g = nx.erdos_renyi_graph(n, 10/(n-1))
# g = nx.watts_strogatz_graph(n, 4, 0.001)
# g = nx.barabasi_albert_graph(n, 2)
# g = nx.powerlaw_cluster_graph(n, 2, 0.01)
# g = nx.relaxed_caveman_graph(16, 16, 0.12)
# g = nx.read_graphml("risk.graphml").to_undirected()
# g = nx.read_graphml("USAir97.graphml").to_undirected()

g = nx.random_geometric_graph(n, 0.125)
# position is stored as node attribute data for random_geometric_graph
pos = nx.get_node_attributes(g, "pos")

#print("Nodes:", len(g.nodes))
#pos = nx.fruchterman_reingold_layout(g, k=0.1)

# The KK layout algorithm takes a minute
# pos = nx.kamada_kawai_layout(g)


#### Pos only seems to be given for random_geometric_graph - Way of getting this attribute for other graphs?

In [ ]:
graph = infectionGrapher(g)

graph.infector(niters=50,case_zero=94,trans_p=0.5,rec_p=0.25,rec_trans_p=0.25)
graph.graphAnimator()
display_animation(graph.graph_ani)

In [ ]:
graph.homologyAnimator(0)
graph.homologyAnimator(1)

In [ ]:
graph.homologyPlot(1,graph.fin_number)

#### The key variable is the probability of transport (trans_p).  Set it too high and life is wiped out very quickly, too low and nothing interesting to report.



#### As an aside it would be interesting to also include more variables, imcluding a resistance to infection parameter that could change as immunity to infection was acheived (either by multiple occurances of avoiding infection or recovering from infection)

#### Quick analysis.  Alternative plot of final infection of the population and then a mapping of rate of infection

## TDA Piece

To suport evaluation of infection spread via topological techniques

#### Ripser.py is a lean persistent homology package for Python. Building on the blazing fast C++ Ripser package as the core computational engine this is my preferred implitation, for reasons of both performance, simplicity of use and supporting material.

https://ripser.scikit-tda.org/


#### Now to produce two plots, one for the distribution of remaining healthy nodes and one for the infected nodes.

#### One annoying factor is RIPS projects into unity space, so healthy and infected nodes appear to overlap.   Early example of how TDA results can lead to mis-interpreation.  Maybe need to scale via original spatial parameters???

#### Key question for future of TDA is there something interesting in the analysis of H1?

Another question: how relevant is H0 for the infected plot to predicting future spread? Worth considering the number of clusters and the relative size of those clusters.

#### So above figures are for analysis of population at end of epoch. We could save data for each time step in experiment, leading to an animation of disease propagation and dynamic analysis.

If we really want to see how this infection evolves topologically, a dynamic graph is a must I'd say.